In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO


In [ ]:
# Dataset Setup

dataset_path = "data"
images_dir = os.path.join(dataset_path, "images")
labels_dir = os.path.join(dataset_path, "labels")

# create train/val/test splits (70/20/10)
all_images = [f for f in os.listdir(images_dir) if f.endswith(".jpg")]
train_imgs, temp_imgs = train_test_split(all_images, test_size=0.3, random_state=42)
val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.33, random_state=42)

def move_files(file_list, split):
    os.makedirs(os.path.join(dataset_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_path, split, "labels"), exist_ok=True)
    for f in file_list:
        shutil.copy(os.path.join(images_dir, f), os.path.join(dataset_path, split, "images", f))
        label_file = f.replace(".jpg", ".txt")
        shutil.copy(os.path.join(labels_dir, label_file), os.path.join(dataset_path, split, "labels", label_file))

move_files(train_imgs, "train")
move_files(val_imgs, "val")
move_files(test_imgs, "test")

print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")


In [ ]:
# Define YAML config

yaml_content = f"""
path: {dataset_path}
train: train/images
val: val/images
test: test/images

nc: 6   # number of defect classes
names: ['missing_component', 'solder_bridge', 'misalignment', 'scratch', 'short_circuit', 'other']
"""

with open(os.path.join(dataset_path, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("YAML config file created!")

In [ ]:
# Train YOLO Model

model = YOLO("yolov8s.pt")
results = model.train(
    data=os.path.join(dataset_path, "data.yaml"),
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    device=0
)